In [1]:
import numpy as np
import pandas as pd
import glob
from newspaper import Article
from time import sleep
from tqdm import tqdm
import time
import signal
import selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
import os

In [2]:
site='https://www.niedziela.pl/archiwum/'

In [8]:
range(2020,2022)

range(2020, 2022)

In [11]:
d = webdriver.Chrome()

eds_links=[]

for yr in range(2020,2023):
    print(yr)
    d.get(site+str(yr))
    sleep(1)
    try:
        d.find_element_by_xpath('//p[@class="fc-button-label"]').click()
        sleep(1)
        d.find_element_by_xpath('//a[@class="closecookies btn btn-label btn-sm"]').click()
    except:
        pass
    
    eds=d.find_elements_by_xpath('//a[@class=""]')

    for ed in eds:
        eds_links.append(ed.get_attribute('href'))

d.quit()

2020


/Users/michalpalinski/anaconda3/envs/myPython/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.
/Users/michalpalinski/anaconda3/envs/myPython/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if sys.path[0] == '':
/Users/michalpalinski/anaconda3/envs/myPython/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  app.launch_new_instance()


2021
2022


In [13]:
eds_clean=[]
for ed in eds_links:
    if 'numer' in ed:
        eds_clean.append(ed)
    else:
        pass        

In [15]:
len(eds_clean)

138

In [123]:
d = webdriver.Chrome()

for ed in eds_clean:
    arts_list=[]
    d.get(ed)
    sleep(1)
    arts=d.find_elements_by_xpath('//div[@class="content-list"]//a')
    for art in arts:
        arts_list.append(art.get_attribute('href'))
    date='_'.join(ed.split('/')[4:6])
    temp=pd.DataFrame(arts_list,columns=[date])
    temp.to_csv('./niedzielapl/links_niedzielapl/'+date+'.csv',index=False)

/Users/michalpalinski/anaconda3/envs/myPython/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  import sys


In [125]:
all_files = glob.glob("./niedzielapl/links_niedzielapl/*.csv")

In [126]:
all_files[0]

'./niedzielapl/links_niedzielapl/2020_52.csv'

In [127]:
class TimeoutException(Exception):   # Custom exception class
    pass

def timeout_handler(signum, frame):   # Custom signal handler
    raise TimeoutException

# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

for file in all_files:
    df=pd.read_csv(file)
    col=df.columns[0]
    df.rename(columns={col:'url'},inplace=True)
    
    df_arts=pd.DataFrame()

    for no, row in tqdm(df.iterrows(), total=df.shape[0]):
        # Start the timer. Once 5 seconds are over, a SIGALRM signal is sent.
        signal.alarm(20)    
        # This try/except loop ensures that 
        #   you'll catch TimeoutException when it's sent.
        try:
            # Whatever your function that might hang
            article = Article(row['url'])
            article.download()
            article.parse()
            title=article.title
            text=article.text
            url=row['url']


            df_arts=df_arts.append({
                            'url': url, 
                            'title': title,
                            'text': text,
                            'col':col
                           }, 
                           ignore_index=True) 


        except:
    #         print('exception')
            title=np.nan
            text=np.nan
            url=row['url']

            df_arts=df_arts.append({
                            'url': url, 
                            'title': title,
                            'text': text,
                           }, 
                           ignore_index=True) 

            continue # continue the for loop if function A takes more than 5 second

        else:
            # Reset the alarm
            signal.alarm(0)

        sleep(1)

    df_arts.to_csv('./niedzielapl/arts_niedzielapl/'+col+'.csv')



  0%|          | 0/106 [00:00<?, ?it/s]

  1%|          | 1/106 [00:02<04:37,  2.64s/it]

  2%|▏         | 2/106 [00:05<04:49,  2.78s/it]

  3%|▎         | 3/106 [00:09<05:03,  2.95s/it]

  4%|▍         | 4/106 [00:11<04:36,  2.71s/it]

  5%|▍         | 5/106 [00:13<04:16,  2.54s/it]

  6%|▌         | 6/106 [00:17<05:07,  3.08s/it]

  7%|▋         | 7/106 [00:21<05:31,  3.35s/it]

  8%|▊         | 8/106 [00:23<04:51,  2.98s/it]

  8%|▊         | 9/106 [00:28<05:24,  3.35s/it]

  9%|▉         | 10/106 [00:30<04:49,  3.01s/it]

 10%|█         | 11/106 [00:32<04:27,  2.82s/it]

 11%|█▏        | 12/106 [00:35<04:32,  2.90s/it]

 12%|█▏        | 13/106 [00:39<04:45,  3.07s/it]

 13%|█▎        | 14/106 [00:42<04:43,  3.09s/it]

 14%|█▍        | 15/106 [00:45<04:56,  3.26s/it]

 15%|█▌        | 16/106 [00:50<05:21,  3.57s/it]

 16%|█▌        | 17/106 [00:53<05:04,  3.42s/it]

 17%|█▋        | 18/106 [00:55<04:35,  3.13s/it]

 18%|█▊        | 19/106 [00:59<04:47,  3.30s/it]

 19%|█▉        |

In [34]:
all_files = glob.glob("./niedzielapl/arts_niedzielapl/*.csv")
for filename in all_files:
    date=filename.split('/')[-1].split('.')[0]
    try:
        df = pd.read_csv(filename, index_col=None, header=0,lineterminator='\n')
    except:
        print(filename)
        pass
    
    df=df.dropna(subset=['text'])
    for idx,row in df.iterrows():
        file = f"./niedzielapl/txt_niedzielapl/{date}/{row['title'].replace('/','')}.txt"
        os.makedirs(os.path.dirname(file), exist_ok=True)
        with open(file, "w+") as f:
            f.write('Title: '+row['title']+'\n'+'\n'+'Link: '+row['url']+'\n'+'\n'+'Edition: '+row['col']+'\n'+'\n'+'Text: '+row['text'])
        f.close()